In [1]:
ip='172.31.19.202'
db='amazon_sp_api'
password='Techblooprint123'
user="blooprint"


In [2]:
from pyspark.sql import SparkSession
import os
from clickhouse_driver import Client

spark = SparkSession.builder \
    .appName("BigCSVWrite") \
    .config("spark.driver.memory", "5g") \
    .config("spark.executor.memory", "6g") \
    .getOrCreate()
spark.conf.set("spark.sql.adaptive.enabled", "true")
spark.conf.set("spark.sql.shuffle.partitions", "50")
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame  # or: from pyspark.sql.dataframe import DataFrame
from pyspark.sql import functions as F, types as T
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
from pyspark.sql.functions import broadcast
from pyspark.sql import functions as F
# Should not raise:
spark.sparkContext._jvm.java.lang.Class.forName("org.postgresql.Driver")

URL  = os.getenv("PG_JDBC_URL", f"jdbc:postgresql://{ip}:5433/{db}")
USER = os.getenv("PG_USER", user)
PWD  = os.getenv("PG_PASSWORD", password)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/15 09:29:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [130]:
portfolio_query = "(SELECT * FROM public.portfolio_id_name_mapping) AS p"

portfolio_id_name_mapping = (spark.read.format("jdbc")
      .option("url", URL)
      .option("user", USER)
      .option("password", PWD)
      .option("driver", "org.postgresql.Driver")
      .option("dbtable", portfolio_query)
      .load())
campaign_query = "(SELECT * FROM public.campaigns) as c"
campaigns = (spark.read.format("jdbc")
      .option("url", URL)
      .option("user", USER)
      .option("password", PWD)
      .option("driver", "org.postgresql.Driver")
      .option("dbtable", campaign_query)
      .load())
competitor_query = "(SELECT * FROM public.competitor_brand_data) as cb"
competitors = (spark.read.format("jdbc")
      .option("url", URL)
      .option("user", USER)
      .option("password", PWD)
      .option("driver", "org.postgresql.Driver")
      .option("dbtable", competitor_query)
      .load())
sb_kw_query = "(SELECT * FROM public.sponsored_brands_search_term) as sb"
sb_kw = (spark.read.format("jdbc")
      .option("url", URL)
      .option("user", USER)
      .option("password", PWD)
      .option("driver", "org.postgresql.Driver")
      .option("dbtable", sb_kw_query)
      .load())
sp_kw_query = "(SELECT * FROM public.sponsored_products_search_term) as sp"
sp_kw = (spark.read.format("jdbc")
      .option("url", URL)
      .option("user", USER)
      .option("password", PWD)
      .option("driver", "org.postgresql.Driver")
      .option("dbtable", sp_kw_query)
      .load())
sd_kw_query = "(SELECT * FROM public.sponsored_display_targeting) as sd"
sd_kw = (spark.read.format("jdbc")
      .option("url", URL)
      .option("user", USER)
      .option("password", PWD)
      .option("driver", "org.postgresql.Driver")
      .option("dbtable", sd_kw_query)
      .load())

targets_query = "(SELECT * FROM public.targets ) as t"
targets = (spark.read.format("jdbc")
      .option("url", URL)
      .option("user", USER)
      .option("password", PWD)
      .option("driver", "org.postgresql.Driver")
      .option("dbtable", targets_query)
      .load())
bid_query="""(SELECT unnamed_subquery.keyword_id,
            unnamed_subquery.bid,
            unnamed_subquery.bid_tym
           FROM ( SELECT bid_values.keyword_id,
                    bid_values.bid,
                    bid_values.updated_date::date AS bid_tym,
                    rank() OVER (PARTITION BY bid_values.keyword_id ORDER BY bid_values.updated_date DESC) AS rank
                   FROM bid_values) unnamed_subquery
          WHERE unnamed_subquery.rank = 1 ) bid"""
bid_df = (spark.read.format("jdbc")
      .option("url", URL)
      .option("user", USER)
      .option("password", PWD)
      .option("driver", "org.postgresql.Driver")
      .option("dbtable", bid_query)
      .load())
ads_query = "(SELECT * FROM public.ads ) as a"
ads_df = (spark.read.format("jdbc")
      .option("url", URL)
      .option("user", USER)
      .option("password", PWD)
      .option("driver", "org.postgresql.Driver")
      .option("dbtable", ads_query)
      .load())

campaign_df=campaigns.select(F.col("campaignId").alias("campaign_id"),
                 F.col("portfolioId").alias("portfolioId"),
                 F.col("name").alias("campaign_name"),
                 ).dropDuplicates(['campaign_id'])

portfolio_df=portfolio_id_name_mapping.select(F.col("portfolioId").alias("portfolioId"),
                                 F.col("name").alias("portfolio_name")).dropDuplicates(['portfolioId'])
portfolio_mapping=campaign_df.join(portfolio_df,on=campaign_df['portfolioId']==portfolio_df['portfolioId'],how='left')
sb_search_term=sb_kw.select(F.col("account_name"),
             F.split(F.col("campaignId").cast("string"),"\\.").getItem(0).alias("campaignId"),
             F.col("campaignName").alias("campaign_name"),
             F.split(F.col("adGroupId").cast("string"),"\\.").getItem(0).alias("ad_group_id"),
             F.col("adGroupName").alias("ad_group_name"),
             F.split(F.col("keywordId").cast("string"),"\\.").getItem(0).alias("keyword_id"),
             F.col("keywordText").alias("keyword"),
             F.col("searchTerm").alias("search_term"),
             F.col("date").cast("date").alias("date"),
             F.col("impressions").alias("impressions"),
             F.col("clicks").alias("clicks"),
             F.col("cost").alias("ad_spend"),
             F.col("sales").alias("ads_sale"),
             F.col("unitsSold").alias("ads_units"),
             F.col("matchType").alias("match_type"),


             )
sd_search_term=sd_kw.select(F.col("account_name"),
             F.split(F.col("campaignId").cast("string"),"\\.").getItem(0).alias("campaignId"),
             F.col("campaignName").alias("campaign_name"),
             F.split(F.col("adGroupId").cast("string"),"\\.").getItem(0).alias("ad_group_id"),
             F.col("adGroupName").alias("ad_group_name"),
             F.split(F.col("targetingId").cast("string"),"\\.").getItem(0).alias("keyword_id"),
             F.col("targetingText").alias("keyword"),
             F.lit("not_applicable").alias("search_term"),
             F.col("date").cast("date").alias("date"),
             F.col("impressions").alias("impressions"),
             F.col("clicks").alias("clicks"),
             F.col("cost").alias("ad_spend"),
             F.col("sales").alias("ads_sale"),
             F.col("unitsSold").alias("ads_units"),
             F.lit("DSP TGT").alias("match_type"),

             )
sp_search_term=sp_kw.select(F.col("account_name"),
             F.split(F.col("campaignId").cast("string"),"\\.").getItem(0).alias("campaignId"),
             F.col("campaignName").alias("campaign_name"),
             F.split(F.col("adGroupId").cast("string"),"\\.").getItem(0).alias("ad_group_id"),
             F.col("adGroupName").alias("ad_group_name"),
             F.split(F.col("keywordId").cast("string"),"\\.").getItem(0).alias("keyword_id"),
             F.col("targeting").alias("keyword"),
             F.col("searchTerm").alias("search_term"),
             F.col("date").cast("date").alias("date"),
             F.col("impressions").alias("impressions"),
             F.col("clicks").alias("clicks"),
             F.col("cost").alias("ad_spend"),
             F.col("sales30d").alias("ads_sale"),
             F.col("unitsSoldSameSku1d").alias("ads_units"),
             F.col("matchType").alias("match_type"),


             )
keyword_df=sp_search_term.unionByName(sb_search_term).unionByName(sd_search_term)


# ads_df=(ads_df.select(
#     F.col("campaignId").alias("campaign_id"),
#     F.col("adProduct").alias("ad_product"),
#     F.col("adType").alias("ad_type"),
#     F.col("state").alias("CAMP STATUS")
# ).dropDuplicates()
# .withColumn("ads_type",
#     F.when(F.col("ad_product")=="SPONSORED_PRODUCTS","SP")
#     .when((F.col("ad_product")=="SPONSORED_BRANDS")&(F.col("ad_type").isin("BRAND_VIDEO","VIDEO")),"SBV")
#     .when((F.col("ad_product")=="SPONSORED_BRANDS")&(F.col("ad_type")=="PRODUCT_COLLECTION"),"SBB")
#     .when((F.col("ad_product")=="SPONSORED_BRANDS")&(F.col("ad_type")=="STORE_SPOTLIGHT"),"SBS")
#     .when((F.col("ad_product")=="SPONSORED_DISPLAY")&(F.col("ad_type")=="IMAGE"),"SDI")
#     .when((F.col("ad_product")=="SPONSORED_DISPLAY")&(F.col("ad_type")=="PRODUCT_AD"),"SDP")
#     .when((F.col("ad_product")=="SPONSORED_DISPLAY")&(F.col("ad_type")=="VIDEO"),"SDV")
#     .otherwise("Not Applicable")          
# ).select(
#     "campaign_id","ads_type","CAMP STATUS"
# )
# )

# ads_df = ads_df.dropDuplicates(["campaign_id"])



# portfolio_mapping = portfolio_mapping.dropDuplicates(["campaign_id"])

# bid_df = bid_df.dropDuplicates(["keyword_id"])



# keyword_df=keyword_df.join(
#     F.broadcast(bid_df),
#     on="keyword_id",
#     how="left"
# )


# portfolio_info=portfolio_mapping.select("portfolio_name","campaign_id",).join(campaign_info ,on ="campaign_id",how="left").withColumnRenamed("campaign_id", "campaignId")
# final_df=keyword_df.join(portfolio_info,on="campaignId",how="left")
# final_df=(
#     final_df
#     .withColumn(
#         "CPC",
#         F.when(F.col("clicks") != 0,
#                F.round(F.col("ad_spend") / F.col("clicks"), 0)
#         ).otherwise(F.lit(0))
#     )
#     .withColumn(
#         "ROI",
#         F.when(F.col("ad_spend") != 0,
#                F.round(F.col("ads_sale") / F.col("ad_spend"), 0)
#         ).otherwise(F.lit(0))
#     )
#     .withColumn(
#         "ACOS",
#         F.when(F.col("ads_sale") != 0,
#                F.round((F.col("ad_spend") * 100) / F.col("ads_sale"), 0)
#         ).otherwise(F.lit(0))
#     )
#     .withColumn(
#         "CVR",
#         F.when(F.col("clicks") != 0,
#                F.round((F.col("ads_units") * 100) / F.col("clicks"), 0)
#         ).otherwise(F.lit(0))
#     )
# )


In [131]:

keyword_df=keyword_df.groupby("account_name","campaignId","ad_group_id","keyword_id").agg(
F.sum("impressions").alias("impressions"),
F.sum("clicks").alias("clicks"),
F.sum("ad_spend").alias("ad_spend"),
F.sum("ads_sale").alias("ads_sale"),
F.sum("ads_units").alias("ads_units")
)


In [132]:
ads_df=(ads_df.select(
    F.col("account_name").alias("ads_account_name"),
    F.col("campaignId").alias("ads_campaign_id"),
    F.col("adProduct").alias("ad_product"),
    F.col("adType").alias("ad_type"),
    F.col("state").alias("CAMP STATUS"),
    F.col("creative_products")
).dropDuplicates()
.withColumn("ads_type",
    F.when(F.col("ad_product")=="SPONSORED_PRODUCTS","SP")
    .when((F.col("ad_product")=="SPONSORED_BRANDS")&(F.col("ad_type").isin("BRAND_VIDEO","VIDEO")),"SBV")
    .when((F.col("ad_product")=="SPONSORED_BRANDS")&(F.col("ad_type")=="PRODUCT_COLLECTION"),"SBB")
    .when((F.col("ad_product")=="SPONSORED_BRANDS")&(F.col("ad_type")=="STORE_SPOTLIGHT"),"SBS")
    .when((F.col("ad_product")=="SPONSORED_DISPLAY")&(F.col("ad_type")=="IMAGE"),"SDI")
    .when((F.col("ad_product")=="SPONSORED_DISPLAY")&(F.col("ad_type")=="PRODUCT_AD"),"SDP")
    .when((F.col("ad_product")=="SPONSORED_DISPLAY")&(F.col("ad_type")=="VIDEO"),"SDV")
    .otherwise("Not Applicable")          
).select(
    "ads_account_name","ads_campaign_id","ads_type","CAMP STATUS","creative_products"
)
)



In [133]:
ads_df = ads_df.dropDuplicates(["ads_campaign_id"])


In [134]:
ads_df.show()

+--------------------+---------------+--------+-----------+--------------------+
|    ads_account_name|ads_campaign_id|ads_type|CAMP STATUS|   creative_products|
+--------------------+---------------+--------+-----------+--------------------+
|Stalwart nutritio...|100118912681214|      SP|    ENABLED|[{"productIdType"...|
|             Detoxie|100373123647857|      SP|    ENABLED|[{"productIdType"...|
|Stalwart nutritio...|100645553381123|      SP|    ENABLED|[{"productIdType"...|
|Stalwart nutritio...| 10118589023770|      SP|    ENABLED|[{"productIdType"...|
|Stalwart nutritio...|101199648918799|      SP|    ENABLED|[{"productIdType"...|
|Stalwart nutritio...|101260849427507|      SP|    ENABLED|[{"productIdType"...|
|            Vansh Co| 10129075998155|     SBV|    ENABLED|[{"productIdType"...|
|Stalwart nutritio...|101411534730219|     SDP|    ENABLED|[{"productIdType"...|
|Stalwart nutritio...|101588492747248|     SDP|    ENABLED|[{"productIdType"...|
|            Vansh Co|101694

In [135]:
keyword_df=keyword_df.join(
    F.broadcast(ads_df),
    (keyword_df["account_name"] == ads_df["ads_account_name"]) &
    (keyword_df["campaignId"] == ads_df["ads_campaign_id"]),
    how="inner"
).drop("ads_account_name","ads_campaign_id")\
.filter(F.col("CAMP STATUS")=="ENABLED")\
    .drop("CAMP STATUS")

In [136]:
schema = T.ArrayType(
    T.StructType([
        T.StructField("productIdType", T.StringType()),
        T.StructField("productId", T.StringType())
    ])
)


In [137]:
keyword_df=keyword_df.withColumn("products", F.from_json(F.col("creative_products"), schema)).drop("creative_products")

In [147]:
(
    keyword_df.withColumn("product", F.explode("products"))
      .select("account_name",
          F.col("product.productId").alias("product_id"),
          F.col("product.productIdType").alias("product_type")
      ).show()
)

+-----------------+------------+------------+
|     account_name|  product_id|product_type|
+-----------------+------------+------------+
|BESTTECHNOLOGIESS|  B0C4HYR7VZ|        ASIN|
|BESTTECHNOLOGIESS|MZ-ECO-2BR-D|         SKU|
|BESTTECHNOLOGIESS|  B0C4HY2JW1|        ASIN|
|BESTTECHNOLOGIESS|MZ-ECO-3BR-D|         SKU|
|BESTTECHNOLOGIESS|  B0C4HY2JW1|        ASIN|
|BESTTECHNOLOGIESS|MZ-ECO-3BR-D|         SKU|
|          awenest|  B0CFBD5QX4|        ASIN|
|          awenest|  B0C4DS6GC1|        ASIN|
|          awenest|  B0CFBD5QX4|        ASIN|
|          awenest|  B0C3W41JXG|        ASIN|
|          awenest|  B0C3W41JXG|        ASIN|
|          awenest|  B0C3W41JXG|        ASIN|
|          awenest|  B0D1KLS6JQ|        ASIN|
|          awenest|  B0DLWPLD1W|        ASIN|
|          awenest|  B0CT32YZ1K|        ASIN|
|          awenest|  B0DKY2ST3Y|        ASIN|
|          awenest|  B0DKY2ST3Y|        ASIN|
|          awenest|  B0C3W41JXG|        ASIN|
|          awenest|  B0C3W41JXG|  